In [25]:
import tensorflow as tf

import tensorflow.contrib.slim as slim
from tqdm import tqdm
import os

In [27]:
os.getpid()

94420

In [35]:
import sys
sys.path.insert(0, '/home/midhun/Github/Modules/')

from alpha import iterate_minibatches

In [17]:
# LeNet-5 model
class Model:
    def __init__(self, data_X, data_y):
        self.n_class = 10
        self._create_architecture(data_X, data_y)

    def _create_architecture(self, data_X, data_y):
        y_hot = tf.one_hot(data_y, depth = self.n_class)
        logits = self._create_model(data_X)
        predictions = tf.argmax(logits, 1, output_type = tf.int32)
        self.loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels = y_hot, 
                                                                              logits = logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(self.loss)
        self.accuracy = tf.reduce_sum(tf.cast(tf.equal(predictions, data_y), tf.float32))

    def _create_model(self, X):
        X1 = X - 0.5
        X1 = tf.pad(X1, tf.constant([[0, 0], [2, 2], [2, 2], [0, 0]]))
        with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                            weights_initializer = tf.truncated_normal_initializer(0.0, 0.1)):
            net = slim.conv2d(X1, 6, [5, 5], padding = 'VALID')
            net = slim.max_pool2d(net, [2, 2])
            net = slim.conv2d(net, 16, [5, 5], padding = 'VALID')
            net = slim.max_pool2d(net, [2, 2])
            
            net = tf.reshape(net, [-1, 400])
            net = slim.fully_connected(net, 120)
            net = slim.fully_connected(net, 84)
            net = slim.fully_connected(net, self.n_class, activation_fn = None)
        return net

In [18]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val     = mnist.validation.images, mnist.validation.labels
X_test, y_test   = mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
X_train.shape, X_test.shape

((55000, 28, 28, 1), (10000, 28, 28, 1))

In [20]:
y_test[:5]

array([7, 2, 1, 0, 4], dtype=uint8)

In [34]:
epochs = 10
batch_size = 64
iterations = len(y_train) * epochs

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# Generate the complete Dataset required in the pipeline
dataset = dataset.repeat(epochs).batch(batch_size)
iterator = dataset.make_one_shot_iterator()

data_X, data_y = iterator.get_next()
data_y = tf.cast(data_y, tf.int32)
model = Model(data_X, data_y)

with tf.Session() as sess, tqdm(total = iterations) as pbar:
    sess.run(tf.global_variables_initializer())

    tot_accuracy = 0
    try:
        while True:
            accuracy, _ = sess.run([model.accuracy, model.optimizer])
            tot_accuracy += accuracy
            pbar.update(batch_size)
    except tf.errors.OutOfRangeError:
        pass

print('\nAverage training accuracy: {:.4f}'.format(tot_accuracy / iterations))

550016it [00:26, 20826.90it/s]                            


Average training accuracy: 0.9825


In [33]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    minibatch_size = 64
    tot_accuracy = 0
    for epoch in tqdm(range(10)):
        for i, minibatch in enumerate(iterate_minibatches(X_train, y_train, minibatch_size)):
            (minibatch_X, minibatch_Y) = minibatch
            accuracy, _ = sess.run([model.accuracy, model.optimizer], feed_dict={data_X:minibatch_X, data_y:minibatch_Y})
            tot_accuracy += accuracy


print('\nAverage training accuracy: {:.4f}'.format(tot_accuracy / iterations))

100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


Average training accuracy: 0.9832
